# Test PDPbox with binary classification problem
## Kaggle Titanic with XGBoost
This demo is based on **Titanic: Machine Learning from Disaster**  
https://www.kaggle.com/c/titanic/details/tutorials

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/Users/sosuneko/Desktop/develop/PDPbox")

In [ ]:
import pandas as pd
from pdpbox import pdp, get_example, info_plots

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# versions
- `xgboost`: 1.3.3
- `matplotlib`: 3.1.1
- `sklearn`: 0.23.1

In [ ]:
import xgboost
from xgboost import XGBClassifier
print(xgboost.__version__)

In [ ]:
import matplotlib
print(matplotlib.__version__)

In [ ]:
import sklearn
print(sklearn.__version__)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# train a simple xgboost classifier
We assume there is a `titanic` data folder under the same directory. The dataset is directly downloaded from kaggle with command `kaggle competitions download -c titanic`. Reference: https://github.com/Kaggle/kaggle-api

In [ ]:
! kaggle competitions download -c titanic

In [ ]:
! ls titanic

In [ ]:
train_df = pd.read_csv('titanic/train.csv')
print(train_df.shape)
display(train_df.head())

## simple feature engineering

In [ ]:
titanic_data = train_df.copy()
titanic_data = titanic_data.drop('Cabin', axis=1)
titanic_data['Age'] = titanic_data['Age'].fillna(titanic_data['Age'].median())
titanic_data['Embarked']= titanic_data['Embarked'].fillna(titanic_data['Embarked'].value_counts().index[0])
LE = LabelEncoder()
titanic_data['Sex'] = LE.fit_transform(titanic_data['Sex'])
for v in ['C', 'S', 'Q']:
    titanic_data['Embarked_{}'.format(v)] = (titanic_data['Embarked'] == v).map(int)

In [ ]:
features= ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_S', 'Embarked_Q']
target = ['Survived']
X_train, X_test, y_train, y_test = train_test_split(
    titanic_data[features], titanic_data[target], test_size = 0.2, random_state=42)

## model training

In [ ]:
%%time
classifier = XGBClassifier(
    max_depth=12,
    subsample=0.33,
    objective='binary:logistic',
    n_estimators=100,
    learning_rate = 0.01)
eval_set = [(X_train,y_train), (X_test,y_test)]

classifier.fit(
    X_train, y_train.values.ravel(), 
    early_stopping_rounds=12, 
    eval_metric=["error", "logloss"],
    eval_set=eval_set, 
    verbose=True
)
classifier.score(X_test,y_test)

In [ ]:
titanic_features = features
titanic_model = classifier
titanic_target = 'Survived'

## get model prediction

# [for internal usage] save for unit test and tutorial

In [ ]:
import joblib
import json

In [ ]:
titanic_data.to_csv('../pdpbox/examples/titanic/titanic_data.csv', index=False)
joblib.dump(titanic_model, '../pdpbox/examples/titanic/titanic_model.pkl')

with open('../pdpbox/examples/titanic/titanic_info.json', 'w') as fout:
    json.dump({
        'features': titanic_features,
        'target': titanic_target
    }, fout, indent=4)

# or you can directory load the data and model

In [ ]:
test_titanic = get_example.titanic()
print(test_titanic.keys())

In [ ]:
titanic_data = test_titanic['data']
titanic_features = test_titanic['features']
titanic_model = test_titanic['xgb_model']
titanic_target = test_titanic['target']

In [ ]:
titanic_data.groupby(["Sex"], as_index=False).count()

# 1. Binary feature: Sex

## 1.1 target distribution through feature 'Sex'

In [ ]:
target_gender = info_plots.TargetPlot(
    df=titanic_data,
    feature="Sex",
    feature_name="gender",
    target=titanic_target,
)

In [ ]:
fig, axes, summary_df = target_gender.plot(
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig.update_layout(xaxis=dict(ticktext=["Female", "Male"]))

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = target_gender.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
)
_ = axes['bar_axes'][0].set_xticklabels(['Female', 'Male'])

## 1.2 check prediction distribution through feature 'Sex'

In [ ]:
predict_gender = info_plots.PredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature="Sex",
    feature_name="gender",
    pred_func=None,
    n_classes=None,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_gender.plot(
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig.update_xaxes(ticktext=["Female", "Male"])

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = predict_gender.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
)
_ = axes['bar_axes'][0].set_xticklabels(['Female', 'Male'])

### pass in `pred_func`

In [ ]:
def titanic_predict_proba(model, X, predict_kwds={}):
    return model.predict_proba(X, **predict_kwds) + 0.05

predict_gender = info_plots.PredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature="Sex",
    feature_name="gender",
    pred_func=titanic_predict_proba,
    n_classes=None,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_gender.plot(
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

## 1.3 pdp for feature 'Sex'

In [ ]:
pdp_sex = pdp.PDPIsolate(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature="Sex",
    feature_name="gender",
)

In [ ]:
fig, axes = pdp_sex.plot(
    center=False,
    plot_lines=True,
    frac_to_plot=100,
    cluster=False,
    n_cluster_centers=None,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=False,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True},
    engine='plotly',
    template='plotly_white',
)
fig.update_xaxes(ticktext=["Female", "Male"], row=2, col=1)

In [ ]:
fig, axes = pdp_sex.plot(
    center=True,
    plot_lines=True,
    frac_to_plot=100,
    cluster=False,
    n_cluster_centers=None,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=False,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True},
    engine='matplotlib',
    template='plotly_white',
)
_ = axes['dist_axes'][0].set_xticklabels(['Female', 'Male'])

# 2. one-hot encoding feature: embarked
for one-hot encoding feature, should provide a list containing all the related columns

## 2.1 target distribution through feature 'embarked'

In [ ]:
target_embarked = info_plots.TargetPlot(
    df=titanic_data,
    feature=['Embarked_C', 'Embarked_S', 'Embarked_Q'],
    feature_name="embarked",
    target=titanic_target,
)

In [ ]:
fig, axes, summary_df = target_embarked.plot(
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = target_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
)

## 2.2 check prediction distribution through feature 'embarked'

In [ ]:
predict_embarked = info_plots.PredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature=['Embarked_C', 'Embarked_S', 'Embarked_Q'],
    feature_name="embarked",
    pred_func=None,
    n_classes=None,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_embarked.plot(
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = predict_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
)

## 2.3 pdp for feature 'embarked'

In [ ]:
pdp_embark = pdp.PDPIsolate(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature=['Embarked_C', 'Embarked_S', 'Embarked_Q'],
    feature_name="embarked",
)

In [ ]:
fig, axes = pdp_embark.plot(
    center=True,
    plot_lines=True,
    frac_to_plot=1,
    cluster=True,
    n_cluster_centers=50,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=False,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True, "line": {"hl_color": "#f46d43"}},
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes = pdp_embark.plot(
    center=False,
    plot_lines=True,
    frac_to_plot=1,
    cluster=True,
    n_cluster_centers=50,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=False,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True, "line": {"hl_color": "#f46d43"}},
    engine='matplotlib',
    template='plotly_white',
)

### pass in `data_transformer`
Since some features are highly correlated.

In [ ]:
for col in ['Embarked_C', 'Embarked_S', 'Embarked_Q']:
    print(col, titanic_data[titanic_data[col] == 1]["Fare"].mean())

In [ ]:
def data_trans(df):
    embarkeds = ['Embarked_C', 'Embarked_S', 'Embarked_Q']
    fares = [60, 28, 13]
    df["Fare"] = df[embarkeds].apply(lambda x: fares[np.argmax(x)], axis=1)
    return df

pdp_embark = pdp.PDPIsolate(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature=['Embarked_C', 'Embarked_S', 'Embarked_Q'],
    feature_name="embarked",
    data_transformer=data_trans,
)

In [ ]:
fig, axes = pdp_embark.plot(
    center=True,
    plot_lines=True,
    frac_to_plot=1,
    cluster=True,
    n_cluster_centers=50,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=False,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True, "line": {"hl_color": "#f46d43"}},
    engine='plotly',
    template='plotly_white',
)
fig

# 3. numeric feature: Fare

## 3.1 target distribution through feature 'Fare'

In [ ]:
target_fare = info_plots.TargetPlot(
    df=titanic_data,
    feature="Fare",
    feature_name="fare",
    target=titanic_target,
    num_grid_points=10,
    grid_type='percentile',
    percentile_range=None,
    grid_range=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoint=True,
)

In [ ]:
fig, axes, summary_df = target_fare.plot(
    show_percentile=True,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = target_fare.plot(
    show_percentile=True,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine="matplotlib",
)

## 3.2 check prediction distribution through feature 'Fare'

In [ ]:
predict_fare = info_plots.PredictPlot(
    model=titanic_model,
    model_features=titanic_features,
    df=titanic_data,
    feature="Fare",
    feature_name="fare",
    pred_func=None,
    n_classes=None,
    num_grid_points=10,
    grid_type='percentile',
    percentile_range=None,
    grid_range=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoint=True,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_fare.plot(
    show_percentile=True,
    figsize=None,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = predict_fare.plot(
    show_percentile=True,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
)

## 3.3 pdp for feature 'Fare'

In [ ]:
pdp_fare = pdp.PDPIsolate(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    feature="Fare",
    feature_name="fare",
    pred_func=None,
    n_classes=None,
    memory_limit=0.5,
    chunk_size=-1,
    n_jobs=1,
    predict_kwds={},
    data_transformer=None,
    cust_grid_points=None,
    grid_type='percentile',
    num_grid_points=10,
    percentile_range=None,
    grid_range=None,
)

In [ ]:
fig, axes = pdp_fare.plot(
    center=True,
    plot_lines=True,
    frac_to_plot=100,
    cluster=False,
    n_cluster_centers=None,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=True,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params={"pdp_hl": True},
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes = pdp_fare.plot(
    center=True,
    plot_lines=True,
    frac_to_plot=100,
    cluster=False,
    n_cluster_centers=None,
    cluster_method='accurate',
    plot_pts_dist=True,
    to_bins=False,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

# 4. Interaction between two variables

## 4.1 'Age' and 'Fare'

### 4.1.1 InterectTargetPlot 

In [ ]:
target_age_fare = info_plots.InterectTargetPlot(
    df=titanic_data,
    features=["Age", "Fare"],
    feature_names=["Age", "Fare"],
    target=titanic_target,
    num_grid_points=10,
    grid_types="percentile",
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
)

In [ ]:
fig, axes, summary_df = target_age_fare.plot(
    show_percentile=True,
    figsize=(1200, 700),
    annotate=True,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df.head()

In [ ]:
fig, axes, summary_df = target_age_fare.plot(
    show_percentile=True,
    figsize=(16, 10),
    dpi=300,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
)

### 4.1.2 InterectPredictPlot

In [ ]:
predict_age_fare = info_plots.InterectPredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", "Fare"],
    feature_names=["Age", "Fare"],
    pred_func=None,
    n_classes=None,
    num_grid_points=10,
    grid_types='percentile',
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_age_fare.plot(
    show_percentile=True,
    figsize=(1200, 800),
    ncols=2,
    annotate=True,
    plot_params={"subplot_ratio": {"y": [10, 1]}},
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df.head()

In [ ]:
fig, axes, summary_df = predict_age_fare.plot(
    show_percentile=True,
    figsize=(16, 10),
    dpi=300,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
)

### 4.1.3 PDPInteract

In [ ]:
pdp_age_fare = pdp.PDPInteract(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", "Fare"],
    feature_names=["age", "fare"],
    pred_func=None,
    n_classes=None,
    memory_limit=0.5,
    chunk_size=-1,
    n_jobs=1,
    predict_kwds={},
    data_transformer=None,
    num_grid_points=10,
    grid_types='percentile',
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
)

In [ ]:
fig, axes = pdp_age_fare.plot(
    plot_type="contour",
    plot_pdp=True,
    to_bins=True,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes = pdp_age_fare.plot(
    plot_type='grid',
    plot_pdp=True,
    to_bins=False,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

## 4.2 'Age' and 'Sex'

### 4.2.1 InterectTargetPlot

In [ ]:
target_age_gender = info_plots.InterectTargetPlot(
    df=titanic_data,
    features=["Age", "Sex"],
    feature_names=["age", "gender"],
    target=titanic_target,
    num_grid_points=10,
    grid_types="percentile",
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
)

In [ ]:
fig, axes, summary_df = target_age_gender.plot(
    show_percentile=True,
    figsize=(1200, 700),
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df.head()

In [ ]:
fig, axes, summary_df = target_age_gender.plot(
    show_percentile=True,
    figsize=None,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.2.2 InterectPredictPlot

In [ ]:
predict_age_gender = info_plots.InterectPredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", "Sex"],
    feature_names=["age", "gender"],
    pred_func=None,
    n_classes=None,
    num_grid_points=10,
    grid_types='percentile',
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_age_gender.plot(
    show_percentile=True,
    figsize=(1200, 700),
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df.head()

In [ ]:
fig, axes, summary_df = predict_age_gender.plot(
    show_percentile=True,
    figsize=None,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.2.3 PDPInteract

In [ ]:
pdp_age_gender = pdp.PDPInteract(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", "Sex"],
    feature_names=["age", "gender"],
    pred_func=None,
    n_classes=None,
    memory_limit=0.5,
    chunk_size=-1,
    n_jobs=1,
    predict_kwds={},
    data_transformer=None,
    num_grid_points=10,
    grid_types='percentile',
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
)

In [ ]:
fig, axes = pdp_age_gender.plot(
    plot_type='contour',
    plot_pdp=True,
    to_bins=True,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes = pdp_age_gender.plot(
    plot_type='grid',
    plot_pdp=True,
    to_bins=True,
    show_percentile=True,
    which_classes=None,
    figsize=None,
    dpi=300,
    ncols=2,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

## 4.3 'Age' and 'Embarked'

### 4.3.1 InterectTargetPlot

In [ ]:
target_age_embarked = info_plots.InterectTargetPlot(
    df=titanic_data,
    features=["Age", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["Age", "Embarked"],
    target=titanic_target,
    num_grid_points=10,
    grid_types="percentile",
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
)

In [ ]:
fig, axes, summary_df = target_age_embarked.plot(
    show_percentile=True,
    figsize=(1200, 700),
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes, summary_df = target_age_embarked.plot(
    show_percentile=True,
    figsize=None,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.3.2 InterectPredictPlot

In [ ]:
predict_age_embarked = info_plots.InterectPredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["Age", "Embarked"],
    pred_func=None,
    n_classes=None,
    num_grid_points=10,
    grid_types='percentile',
    percentile_ranges=None,
    grid_ranges=None,
    cust_grid_points=None,
    show_outliers=False,
    endpoints=True,
    predict_kwds={},
    chunk_size=-1,
)

In [ ]:
fig, axes, summary_df = predict_age_embarked.plot(
    show_percentile=True,
    figsize=(1200, 700),
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes, summary_df = predict_age_embarked.plot(
    show_percentile=True,
    figsize=None,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.3.3 PDPInteract

In [ ]:
pdp_embark_age = pdp.PDPInteract(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Age", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["Age", "Embarked"],
)

In [ ]:
fig, axes = pdp_embark_age.plot(
    plot_type="grid",
    plot_pdp=True,
    show_percentile=True,
    engine="plotly",
    template="plotly_white",
)
fig

In [ ]:
fig, axes = pdp_embark_age.plot(
    plot_type="grid",
    plot_pdp=True,
    show_percentile=True,
    engine="matplotlib",
    template="plotly_white",
)

## 4.4 'Sex' and 'Embarked'

### 4.4.1 InterectTargetPlot

In [ ]:
target_gender_embarked = info_plots.InterectTargetPlot(
    df=titanic_data,
    features=["Sex", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["gender", "embarked"],
    target=titanic_target,
)

In [ ]:
fig, axes, summary_df = target_gender_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
summary_df

In [ ]:
fig, axes, summary_df = target_gender_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.4.2 InterectPredictPlot

In [ ]:
predict_gender_embarked = info_plots.InterectPredictPlot(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Sex", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["gender", "embarked"],
)

In [ ]:
fig, axes, summary_df = predict_gender_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='plotly',
    template='plotly_white',
)
fig

In [ ]:
fig, axes, summary_df = predict_gender_embarked.plot(
    figsize=None,
    dpi=300,
    ncols=2,
    annotate=True,
    plot_params=None,
    engine='matplotlib',
    template='plotly_white',
)

### 4.4.3 PDPInteract

In [ ]:
pdp_gender_embark = pdp.PDPInteract(
    model=titanic_model,
    df=titanic_data,
    model_features=titanic_features,
    features=["Sex", ['Embarked_C', 'Embarked_S', 'Embarked_Q']],
    feature_names=["gender", "embarked"],
)

In [ ]:
fig, axes = pdp_gender_embark.plot(
    plot_type="grid",
    plot_pdp=True,
    engine="plotly",
    template="plotly_white",
)
fig

In [ ]:
fig, axes = pdp_gender_embark.plot(
    plot_type="grid",
    plot_pdp=True,
    engine="matplotlib",
    template="plotly_white",
)